In [1]:
import face_recognition
import os
from PIL import Image
import cv2


%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [14]:
image_path = os.getenv('HOME')+'/aiffel/face_embedding/celeb_images/trump.jpg'
dir_path = os.getenv('HOME')+'/aiffel/face_embedding/celeb_images'
path_to_save = os.getenv('HOME')+'/aiffel/face_embedding/Conv2PID_images'

In [15]:
def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]

    return cropped_face

In [16]:
def get_gropped_face_save(dir_path, path_to_save):
    file_list = os.listdir(dir_path)
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_gropped_face(img_path)
        pillow_image = Image.fromarray(face)
        pillow_image.save(path_to_save + os.path.splitext(file)[0] + '.' + 'jpg')
            
get_gropped_face_save(dir_path, path_to_save)

IndexError: list index out of range

In [5]:
def get_face_embedding(face):
    return face_recognition.face_encodings(face)


In [6]:
def get_face_embedding_dict(path_to_save):
    file_list = os.listdir(path_to_save)
    embedding_dict = {}
    for file in file_list:
        img_path = os.path.join(path_to_save, file)
        face = mpimg.imread(img_path)
        embedding = get_face_embedding(face)
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
                    # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
    return embedding_dict

In [9]:
embedding_dict = get_face_embedding_dict(path_to_save)

In [ ]:
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

In [ ]:
# name1과 name2의 거리를 비교하는 함수를 생성하되, name1은 미리 지정하고, name2는 호출시에 인자로 받도록 합니다.
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

# 이렇게 생성된 함수 sort_key_func는 sort_key_func('obama') 라고 호출할 때 trump와 obama 사이의 임베딩 벡터 거리를 계산합니다.

In [ ]:
def get_nearest_face(name, top=4):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))

    for i in range(top+1):
        if i == 0 :   # 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))
    return sorted_faces, sort_key_func,